<a href="https://colab.research.google.com/github/geopan2000/Store-Sales-Forecasting/blob/main/StoreSalesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

# Clone the GitHub repository
!git clone https://github.com/geopan2000/Store-Sales-Forecasting.git

# Change directory to the cloned repository
%cd Store-Sales-Forecasting


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'Store-Sales-Forecasting'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 232.50 KiB | 2.98 MiB/s, done.
/content/Store-Sales-Forecasting/Store-Sales-Forecasting/Store-Sales-Forecasting


In [8]:
import pandas as pd

In [9]:
holidays_events_df = pd.read_csv('/content/drive/MyDrive/StoreSalesForecasting/holidays_events.csv')
oil_df = pd.read_csv('/content/drive/MyDrive/StoreSalesForecasting/oil.csv')
sample_submission_df = pd.read_csv('/content/drive/MyDrive/StoreSalesForecasting/sample_submission.csv')
stores_df = pd.read_csv('/content/drive/MyDrive/StoreSalesForecasting/stores.csv')
test_df = pd.read_csv('/content/drive/MyDrive/StoreSalesForecasting/test.csv')
train_df = pd.read_csv('/content/drive/MyDrive/StoreSalesForecasting/train.csv')
transactions_df = pd.read_csv('/content/drive/MyDrive/StoreSalesForecasting/transactions.csv')

In [16]:
sample_submission_df.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


In [15]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [14]:
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [17]:
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [18]:
test_df.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [20]:
transactions_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
